<a href="https://colab.research.google.com/github/ganesh3/pytorch-work/blob/master/mnist_resnet_pytorch_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

     |████████████████████████████████| 409kB 5.6MB/s 
     |████████████████████████████████| 276kB 17.2MB/s 
     |████████████████████████████████| 829kB 27.4MB/s 
     |████████████████████████████████| 2.8MB 49.0MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=fba31886997aba4a036d51f285a0b1eadc235825633c19b430fbf70f4e826958
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=12d386043e96e661ef5de6df6504f5d161c70c5d03970556aca33b1b84c3016d
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML future
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.0 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

In [24]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy

In [43]:
class Resnet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28 * 28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h2 + h1)
    logits = self.l3(do)
    return logits

  def configure_optimizers(self):
    optimizer =  optim.SGD(self.parameters(), lr=1e-2)
    return optimizer

  def training_step(self, batch, batch_idx):
    x, y = batch
    
    b = x.size(0)
    x = x.view(b, -1)
    
    logit = self(x)
    
    J = self.loss(logit, y)

    acc = accuracy(logit, y)
    pbar = {'train_accuracy': acc}

    return {'loss': J, 'progress_bar': pbar}

  def validation_step(self, batch, batch_idx):
    results = self.training_step(batch, batch_idx)
    results['progress_bar']['val_acc'] = results['progress_bar']['train_accuracy']
    del results['progress_bar']['train_accuracy']
    return results

  def validation_epoch_end(self, val_step_outputs):
    avg_val_loss = torch.tensor([x['loss'] for x in val_step_outputs]).mean()
    avg_val_acc = torch.tensor([x['progress_bar']['val_acc'] for x in val_step_outputs]).mean()
    pbar = {'avg_val_acc': avg_val_acc}
    return {'val_loss': avg_val_loss, 'progress_bar': pbar}

  def prepare_data(self):
    datasets.MNIST('data', train=True, download=True, transform = transforms.ToTensor())

  def setup(self):
    dataset = datasets.MNIST('data', train=True, download=False, transform = transforms.ToTensor())
    self.train, self.val = random_split(dataset, [55000, 5000])
  
  def train_dataloader(self):
    train_loader = DataLoader(self.train, batch_size=32)
    return train_loader

  def val_dataloader(self):
    val_loader = DataLoader(self.val, batch_size=32)
    return val_loader

model = Resnet()

In [44]:
trainer = pl.Trainer(progress_bar_refresh_rate=20, max_epochs=5, gpus=8, num_nodes=32)
trainer.fit(model)

MisconfigurationException: ignored

In [32]:
!ls lightning_logs/version_12/checkpoints/

'epoch=4.ckpt'
